In [19]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.transforms
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.nn.functional as F
import scipy

In [20]:
training_data = datasets.MNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor(),
)

test_data = datasets.MNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor(),
)

In [21]:
batch_size = 1

training_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


for X, y in test_dataloader:
    print(f'Shape of X: [N, C, H, W]: {X.shape}')
    print(f'Shape of y: {y.shape}{y.type}')
    
    break

Shape of X: [N, C, H, W]: torch.Size([1, 1, 28, 28])
Shape of y: torch.Size([1])<built-in method type of Tensor object at 0x7f6fcb4cdad0>


In [22]:
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
print(f'Using: {device} device')

Using: cpu device


In [23]:
class LeNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)

        self.fc1 = nn.Linear(400, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1

        for s in size:
            num_features *= s
        
        return num_features

model = LeNet()
print(model)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=10, bias=True)
)


In [24]:
epochs = 10
learning_rate = 1e-3

loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [25]:
def train(dataloader, model, loss_fn, optimzer):
    model.train()

    for X,y in tqdm(dataloader):

        pred = model(X)
        loss = loss_fn(pred, y)

        optimzer.zero_grad()
        loss.backward()
        optimzer.step()

In [26]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    model.eval()

    correct = 0

    with torch.no_grad():

        for X, y in tqdm(dataloader):
            pred = model(X)

            x = (pred.argmax(1) == y)
            x = x.type(torch.float)
            x = x.sum()
            x = x.item()

            correct += x
    print(f'Accuracy(%): {correct/size*100}')

In [27]:
for i in range(epochs):
    train(training_dataloader, model, loss_fn, optim)
    test(test_dataloader, model, loss_fn)
    print(i)

  2%|▏         | 967/60000 [00:05<05:55, 166.16it/s]


KeyboardInterrupt: 